In [1]:
import os

import boto3
import numpy as np
import pandas as pd

from file_utilities import s3_download 


In [2]:
s3 = boto3.client("s3")
bucket = "capstone-bucket-4-friends"


In [5]:
path = "Index/snp500constituents2023.csv"
file_path = s3_download(path)
sp500_members = pd.read_csv(file_path)
sp500_members.head()

snp500constituents2023.csv
../../data/snp500constituents2023.csv


,PERMNO,Company Name,Ticker,SP500 Start,SP500 End
0,10104,ORACLE CORP,ORCL,"Aug. 3, 1989","Dec. 29, 2023"
1,10107,MICROSOFT CORP,MSFT,"June 7, 1994","Dec. 29, 2023"
2,10138,T ROWE PRICE GROUP INC,TROW,"Oct. 13, 1999","Dec. 29, 2023"
3,10145,HONEYWELL INTERNATIONAL INC,HON,"Dec. 31, 1925","Dec. 29, 2023"
4,10516,ARCHER DANIELS MIDLAND CO,ADM,"July 30, 1981","Dec. 29, 2023"


In [33]:
index_constituents = pd.read_csv(index_path, parse_dates=True)

index_constituents.head

<bound method NDFrame.head of      PERMNO                 Company Name Ticker     SP500 Start      SP500 End
0     10104                  ORACLE CORP   ORCL    Aug. 3, 1989  Dec. 29, 2023
1     10107               MICROSOFT CORP   MSFT    June 7, 1994  Dec. 29, 2023
2     10138       T ROWE PRICE GROUP INC   TROW   Oct. 13, 1999  Dec. 29, 2023
3     10145  HONEYWELL INTERNATIONAL INC    HON   Dec. 31, 1925  Dec. 29, 2023
4     10516    ARCHER DANIELS MIDLAND CO    ADM   July 30, 1981  Dec. 29, 2023
..      ...                          ...    ...             ...            ...
498   93096      DOLLAR GENERAL CORP NEW     DG    Dec. 3, 2012  Dec. 29, 2023
499   93132                 FORTINET INC   FTNT   Oct. 11, 2018  Dec. 29, 2023
500   93246         GENERAC HOLDINGS INC   GNRC  March 22, 2021  Dec. 29, 2023
501   93429   C B O E GLOBAL MARKETS INC   CBOE   March 1, 2017  Dec. 29, 2023
502   93436                    TESLA INC   TSLA   Dec. 21, 2020  Dec. 29, 2023

[503 rows x 5 columns

In [34]:
index_constituents["SP500 Start"] = pd.to_datetime(
    index_constituents["SP500 Start"], format="mixed"
)
index_constituents["SP500 End"] = pd.to_datetime(
    index_constituents["SP500 End"], format="mixed"
)

In [38]:
index_start = pd.Timestamp("2023-01-01")
index_end = pd.Timestamp("2023-12-29")

index_ids = index_constituents[
    (index_constituents["SP500 Start"] < index_start)
    & (index_constituents["SP500 End"] == index_end)
]

## filter the crsp_2023_clean

In [3]:
file_path = s3_download("CRSP/crsp_2023_clean.csv")
df = pd.read_csv(file_path)
df["date"] = pd.to_datetime(df["date"])

In [4]:
df.dtypes

PERMNO             int64
date      datetime64[ns]
PRC              float64
RET              float64
dtype: object

In [5]:
df.groupby("PERMNO").count()

,date,PRC,RET
PERMNO,,,
10026,250,250,250
10028,250,250,250
10032,250,250,250
10044,250,250,250
10065,250,250,250
...,...,...,...
93426,250,250,250
93427,250,250,250
93429,250,250,250


In [31]:
index_path = "../data/snp500constituents2023.csv"
s3.download_file(
    f"{bucket}",
    "Index/snp500constituents2023.csv",
    index_path,
)

In [32]:
np.random.seed(1234)
# sampled_permno = df["PERMNO"].sample(50).tolist()
sampled_permno = df["PERMNO"].sample(50).tolist()

In [33]:
index_constituents = pd.read_csv(index_path, parse_dates=True)

index_constituents.head

<bound method NDFrame.head of      PERMNO                 Company Name Ticker     SP500 Start      SP500 End
0     10104                  ORACLE CORP   ORCL    Aug. 3, 1989  Dec. 29, 2023
1     10107               MICROSOFT CORP   MSFT    June 7, 1994  Dec. 29, 2023
2     10138       T ROWE PRICE GROUP INC   TROW   Oct. 13, 1999  Dec. 29, 2023
3     10145  HONEYWELL INTERNATIONAL INC    HON   Dec. 31, 1925  Dec. 29, 2023
4     10516    ARCHER DANIELS MIDLAND CO    ADM   July 30, 1981  Dec. 29, 2023
..      ...                          ...    ...             ...            ...
498   93096      DOLLAR GENERAL CORP NEW     DG    Dec. 3, 2012  Dec. 29, 2023
499   93132                 FORTINET INC   FTNT   Oct. 11, 2018  Dec. 29, 2023
500   93246         GENERAC HOLDINGS INC   GNRC  March 22, 2021  Dec. 29, 2023
501   93429   C B O E GLOBAL MARKETS INC   CBOE   March 1, 2017  Dec. 29, 2023
502   93436                    TESLA INC   TSLA   Dec. 21, 2020  Dec. 29, 2023

[503 rows x 5 columns

In [34]:
index_constituents["SP500 Start"] = pd.to_datetime(
    index_constituents["SP500 Start"], format="mixed"
)
index_constituents["SP500 End"] = pd.to_datetime(
    index_constituents["SP500 End"], format="mixed"
)

In [38]:
index_start = pd.Timestamp("2023-01-01")
index_end = pd.Timestamp("2023-12-29")

index_ids = index_constituents[
    (index_constituents["SP500 Start"] < index_start)
    & (index_constituents["SP500 End"] == index_end)
]

,PERMNO,Company Name,Ticker,SP500 Start,SP500 End
0,10104,ORACLE CORP,ORCL,1989-08-03,2023-12-29
1,10107,MICROSOFT CORP,MSFT,1994-06-07,2023-12-29
2,10138,T ROWE PRICE GROUP INC,TROW,1999-10-13,2023-12-29
3,10145,HONEYWELL INTERNATIONAL INC,HON,1925-12-31,2023-12-29
4,10516,ARCHER DANIELS MIDLAND CO,ADM,1981-07-30,2023-12-29
...,...,...,...,...,...
498,93096,DOLLAR GENERAL CORP NEW,DG,2012-12-03,2023-12-29
499,93132,FORTINET INC,FTNT,2018-10-11,2023-12-29
500,93246,GENERAC HOLDINGS INC,GNRC,2021-03-22,2023-12-29
501,93429,C B O E GLOBAL MARKETS INC,CBOE,2017-03-01,2023-12-29


In [40]:
df_sample = df[df["PERMNO"].isin(index_ids['PERMNO'])]
df_pivot = df_sample.pivot(index="date", columns="PERMNO", values="RET")
df_pivot.head()

PERMNO,10104,10107,10138,10145,10516,10696,11308,11403,11404,11600,...,92611,92614,92655,93002,93089,93096,93132,93246,93429,93436
date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,0.024223,-0.001001,0.000825,0.000000,-0.034464,0.001385,-0.010376,-0.006225,0.004721,0.026696,...,-0.001781,0.011022,-0.021766,-0.010105,0.005215,0.001909,-0.007568,-0.029307,0.006854,-0.122422
2023-01-04,0.009078,-0.043743,0.020247,-0.019879,-0.036364,0.008398,-0.000477,-0.002568,0.008667,0.033344,...,0.025170,0.019922,-0.027264,0.012214,0.016409,-0.013051,-0.004534,0.043087,-0.014723,0.051249
2023-01-05,-0.002012,-0.029638,-0.028287,-0.026519,-0.025003,-0.026063,-0.011443,-0.015261,-0.013459,-0.020426,...,-0.007055,-0.015906,-0.028821,-0.009317,-0.014480,0.007885,-0.017598,-0.035812,-0.005624,-0.029039
2023-01-06,0.016012,0.011785,0.037704,0.028366,0.020420,0.027968,0.019293,0.018240,0.026655,0.010275,...,0.031453,0.023211,0.000082,0.060196,0.029498,0.012794,0.026133,0.025237,-0.002262,0.024651
2023-01-09,0.012608,0.009736,0.022798,0.001902,-0.006748,-0.001077,-0.012461,0.039208,-0.001124,-0.013162,...,0.003904,0.001137,0.000122,-0.019611,-0.004539,-0.030214,0.014787,0.039603,-0.010608,0.059349


In [41]:
date_cutoff = pd.Timestamp("2023-10-01")

In [42]:
train = df_pivot[:date_cutoff]
train_span = len(train)
train_span

187

In [43]:
test = df_pivot[date_cutoff:]
test_span = len(test)
test_span

63

In [44]:
train.index

DatetimeIndex(['2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',
               '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12',
               '2023-01-13', '2023-01-17',
               ...
               '2023-09-18', '2023-09-19', '2023-09-20', '2023-09-21',
               '2023-09-22', '2023-09-25', '2023-09-26', '2023-09-27',
               '2023-09-28', '2023-09-29'],
              dtype='datetime64[ns]', name='date', length=187, freq=None)

Reference: https://github.com/zhuodannychen/Portfolio-Optimization